# 1. Contexto y Objetivo del Proyecto

## 1.1 Contexto del negocio

Una entidad bancaria de origen indio busca **mejorar la toma de decisiones estratégicas** utilizando sus datos históricos de clientes y transacciones.

Si bien el banco dispone de información detallada sobre **clientes, transacciones y perfiles de valor**, actualmente no cuenta con un sistema predictivo robusto que le permita:

- Detectar de forma temprana **transacciones fraudulentas**.  
- Identificar **clientes de alto valor** para estrategias de retención.  
- Segmentar clientes según su **recencia, frecuencia y valor monetario (RFM)**.  
- Estimar el **valor futuro de las transacciones** para la planificación financiera.

Esta falta de analítica avanzada genera **riesgos financieros** (fraudes no detectados) y pérdida de **oportunidades de negocio** (clientes valiosos no priorizados).

En este informe nos concentramos en el componente de **detección de fraude transaccional**.

---

## 1.2 Problema a resolver

El banco necesita un modelo capaz de **distinguir entre transacciones legítimas y fraudulentas**, apoyándose en señales de comportamiento del cliente y en indicadores de riesgo ya existentes en sus sistemas.

De forma específica, se plantea que la probabilidad de que una transacción sea fraudulenta (`is_fraud`) está influida por:

- **Montos atípicos por ubicación** (`AmountZScoreByLocation`), que se desvían del patrón normal del cliente.
- Un **RiskScore alto**, que resume comportamiento histórico y características de riesgo.
- La detección previa de **anomalías de comportamiento** (`IsAnomaly`) por parte del motor de reglas.
- El **momento temporal** de la transacción, en particular:
  - Operar de madrugada (`IsLateNight`),
  - Operar en fin de semana (`IsWeekend`).

En simples palabras, el banco quiere responder a la pregunta:

> **¿Qué señales ayudan al sistema a detectar cuándo una transacción podría ser fraude?**

---

## 1.3 Hipótesis y variables del modelo de clasificación

La hipótesis central del modelo es que ciertas características de riesgo aumentan la probabilidad de que una transacción sea marcada como fraude.

| Tipo          | Variable                   | Descripción                                                      | Relación esperada con Fraude |
| ------------- | -------------------------- | ---------------------------------------------------------------- | ---------------------------- |
| Dependiente   | **is_fraud**               | Etiqueta final: 1 = fraude, 0 = no fraude.                       | —                            |
| Independiente | **AmountZScoreByLocation** | Qué tan anómalo es el monto respecto a la ubicación del cliente. | Positiva                     |
| Independiente | **RiskScore**              | Puntuación de riesgo derivada de características históricas.     | Positiva                     |
| Independiente | **IsAnomaly**              | Indica si el motor de reglas detectó comportamiento atípico.     | Positiva fuerte              |
| Independiente | **IsLateNight**            | Señala si ocurrió de madrugada.                                  | Leve–Moderada positiva       |
| Independiente | **IsWeekend**              | Indica si ocurrió en fin de semana.                              | Leve positiva                |

El objetivo del análisis es **evaluar qué tan bien estas variables permiten diferenciar transacciones legítimas de fraudulentas**, y cuantificar su impacto dentro de los modelos de clasificación que se utilizarán posteriormente.


## 2. Resumen de Modelos Supervisados (EP2)

A continuación se resumen los cinco modelos de clasificación entrenados durante la etapa supervisada, antes de incorporar la variable `cluster_id`. Todos los modelos fueron evaluados con validación cruzada (F1-CV) y posteriormente con un conjunto de prueba (F1 Test, Precisión y Recall). La clase de interés es **fraude (1)**.

| Modelo                  | F1 (CV)          | AUC   | F1 Test | Precisión | Recall  |
|------------------------|------------------|-------|---------|-----------|---------|
| **DecisionTreeClassifier** | 0.113 ± 0.001 | 0.921 | 0.125 | 0.067 | **0.901** |
| **KNeighborsClassifier**   | 0.028 ± 0.002 | 0.581 | 0.029 | 0.019 | 0.059 |
| **MLPClassifier**          | 0.121 ± 0.003 | 0.927 | 0.120 | 0.064 | **0.908** |
| **RandomForestClassifier** | 0.122 ± 0.001 | 0.902 | 0.123 | 0.066 | 0.845 |
| **XGBClassifier**          | 0.057 ± 0.003 | 0.928 | 0.058 | 0.030 | **0.970** |



### Interpretación de resultados

Los cinco modelos presentan comportamientos distintos al momento de detectar la clase minoritaria (fraude), lo cual es esperable debido al fuerte desbalance del dataset.

- **XGBoost** obtuvo el **mayor recall (0.970)** entre todas las opciones, detectando prácticamente todos los fraudes reales. Aunque su precisión es baja (0.03), este comportamiento es adecuado para escenarios donde es preferible generar alertas adicionales antes que dejar pasar un fraude.
- **MLPClassifier** y **DecisionTreeClassifier** también alcanzan altos niveles de recall (0.90), lo que los convierte en modelos efectivos para maximizar la detección, aunque con menor estabilidad que XGBoost.
- **RandomForestClassifier** muestra un buen equilibrio general, pero con menor recall que los modelos anteriores.
- **KNN** es el modelo con menor desempeño, evidenciado por su bajo AUC (0.58), bajo recall (0.059) y F1 muy reducido.

En conjunto, estos resultados muestran que los modelos basados en árboles y métodos gradientes —en particular **XGBoost**— son los más adecuados para el problema de detección de fraude. Por este motivo, XGBoost fue seleccionado como modelo base para análisis posteriores e integración con nuevas variables, como el `cluster_id` proveniente del aprendizaje no supervisado.


## 3. Descripción de los Clusters (K-Means k=6)

Luego de seleccionar K-Means con k = 6 como modelo final de clustering, se analizó la composición de cada grupo utilizando las variables de comportamiento y la proporción de transacciones fraudulentas dentro de cada cluster.

### Distribución de fraude por cluster

Los resultados mostraron diferencias claras entre los segmentos:

| Cluster | Transacciones | Fraudes | % Fraude | Interpretación |
|--------|---------------|---------|----------|----------------|
| **2** | 318 | 49 | **15.41%** | **Cluster de máximo riesgo**: transacciones muy inusuales, montos altamente atípicos. |
| **5** | 3871 | 163 | **4.21%** | **Riesgo alto**: actividad irregular y señales temporales de riesgo. |
| **3** | 6929 | 236 | **3.41%** | **Riesgo medio-alto**: anomalías frecuentes. |
| **4** | 3345 | 30 | **0.89%** | Riesgo bajo, comportamiento moderado. |
| **1** | 10443 | 8 | **0.07%** | Comportamiento casi normal. |
| **0** | 25094 | 8 | **0.03%** | **Cluster más seguro**, transacciones totalmente regulares. |

### Interpretación general

- El **cluster 2** es el más crítico: presenta una tasa de fraude extremadamente alta (15%), lo que indica un comportamiento completamente distinto respecto al resto del dataset.
- Los **clusters 5 y 3** conforman un segundo segmento relevante: aunque no tan extremos como el cluster 2, mantienen proporciones anómalamente altas de fraude.
- Los **clusters 1 y 0** representan la gran mayoría del volumen y presentan un comportamiento estable, con niveles casi nulos de fraude.
- Esta estructura evidencia que el clustering logra separar distintos patrones operativos del banco, y que los grupos extremos (2, 3 y 5) contienen señales fuertes de riesgo que no estaban explícitas en las variables originales.

### Valor para el modelo supervisado

La etiqueta `cluster_id` captura patrones complejos que no estaban en las variables originales.  
Por ejemplo:

- Clientes o transacciones que **operan fuera de sus patrones habituales**,  
- Comportamientos agrupados que solo aparecen cuando se reduce dimensionalidad y se agrupan tendencias,  
- Segmentos altamente fraudulentos que no se detectaban linealmente.

Por esto, `cluster_id` se incorpora como **nueva variable categórica** dentro del modelo XGBoost final y se evalúa su impacto predictivo en la siguiente sección.


## 4. Integración del Clustering en el Modelo Supervisado

En esta etapa se utiliza la nueva variable `cluster_id`, generada a partir del análisis no supervisado, como feature adicional para mejorar la capacidad predictiva del modelo de fraude.

El objetivo es evaluar si la estructura descubierta por K-Means aporta información útil para predecir la probabilidad de fraude en cada transacción.


In [7]:
# =========================================================
# 0) Imports (además del %load_ext que ya tienes arriba)
# =========================================================
%load_ext kedro.ipython
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from xgboost import XGBClassifier
from sklearn.model_selection import (
    StratifiedKFold,
    cross_val_score,
    train_test_split,
    RandomizedSearchCV)

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay)

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


In [8]:
# =========================================================
# 1) Cargar dataset con cluster_id
# =========================================================
df = catalog.load("Features_clustering_v1").copy()
df.head()

[11/27/25 12:51:33] INFO     Loading data from Features_clustering_v1 (ParquetDataset)...      ]8;id=387912;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=521639;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\core.py:277 in load      │
│                                                                                                  │
│    274 │   │   │   self._logger.debug("Loading %s", str(self))                                   │
│    275 │   │   │                                                                                 │
│    276 │   │   │   try:                                                                          │
│ ❱  277 │   │   │   │   return load_func(self)                                                    │
│    278 │   │   │   except DatasetError:                                                          │
│    279 │   │   │   │   raise                                                                     │
│    280 │   │   │   except Exception as exc:                                                      │
│                                                                                                  │
│ C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro_datasets\pandas\parquet_dat │
│ aset.py:177 in load                                                                              │
│                                                                                                  │
│   174 │   │   │   # (<urlopen error file not on local host>),                                    │
│   175 │   │   │   # so we don't join that back to the filepath;                                  │
│   176 │   │   │   # storage_options also don't work with local paths                             │
│ ❱ 177 │   │   │   return pd.read_parquet(load_path, **self._load_args)                           │
│   178 │   │                                                                                      │
│   179 │   │   load_path = f"{self._protocol}{PROTOCOL_DELIMITER}{load_path}"                     │
│   180 │   │   return pd.read_parquet(                                                            │
│                                                                                                  │
│ C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\pandas\io\parquet.py:669 in       │
│ read_parquet                                                                                     │
│                                                                                                  │
│   666 │   │   use_nullable_dtypes = False                                                        │
│   667 │   check_dtype_backend(dtype_backend)                                                     │
│   668 │                                                                                          │
│ ❱ 669 │   return impl.read(                                                                      │
│   670 │   │   path,                                                                              │
│   671 │   │   columns=columns,                                                                   │
│   672 │   │   filters=filters,                                                                   │
│                                                                                                  │
│ C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\pandas\io\parquet.py:258 in read  │
│                                                                                                  │
│   255 │   │   manager = _get_option("mode.data_manager", silent=True)                            │
│   256 │   │   if manager == "array":                                                             │
│   257 │   │   │   to_pandas_kwargs["split_blocks"] = True                                        │
│ ❱ 258 │   │   path_or_handle, handles, filesystem = _get_path_or_handle(                         │
│   259 │   │   │   path,                                    

In [ ]:
# =========================================================
# 2) Matriz de correlación (solo para visualizar)
# =========================================================
plt.figure(figsize=(10, 6))
corr = df.corr()
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Matriz de Correlación - Features_clustering_v1")
plt.show()

# =========================================================
# 3) Preparar datos (X incluye cluster_id)
# =========================================================
X = df.drop("is_fraud", axis=1)
y = df["is_fraud"]

# =========================================================
# 4) Modelo BASE: SMOTE + XGBoost (sin tuning)
# =========================================================
smote = SMOTE(random_state=42)

xgb_base = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=10,   # peso para la clase fraude
    eval_metric="auc",
    random_state=42,
    n_jobs=-1
)

pipe_base = Pipeline([
    ("smote", smote),
    ("model", xgb_base)
])

# --- Validación cruzada ---
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(pipe_base, X, y, cv=cv, scoring="f1")

print("=== MODELO BASE XGBoost + SMOTE ===")
print("F1 promedio (CV):", scores.mean())
print("Desviación estándar:", scores.std())

# --- Train/Test ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

pipe_base.fit(X_train, y_train)
y_pred = pipe_base.predict(X_test)

print("\nReporte de clasificación – MODELO BASE")
print(classification_report(y_test, y_pred, digits=3))

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=["No Fraude", "Fraude"])
disp.plot(cmap="Blues")
plt.title("Matriz de Confusión – XGBoost + SMOTE (BASE)")
plt.show()

# =========================================================
# 5) BÚSQUEDA EXPRESS DE HIPERPARÁMETROS
# =========================================================
# Ratio de clases para scale_pos_weight
neg, pos = (y == 0).sum(), (y == 1).sum()
ratio = neg / pos
print("\nRatio clases (neg/pos):", ratio)

smote = SMOTE(random_state=42)

xgb = XGBClassifier(
    eval_metric="auc",
    random_state=42,
    n_jobs=-1,
    tree_method="approx"   # más rápido
)

pipe = Pipeline([
    ("smote", smote),
    ("model", xgb)
])

param_distributions = {
    "model__n_estimators": [200, 300],
    "model__learning_rate": [0.05, 0.1],
    "model__max_depth": [3, 4, 5],
    "model__subsample": [0.8, 1.0],
    "model__colsample_bytree": [0.8, 1.0],
    "model__scale_pos_weight": [ratio/3, ratio/4]
}

cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

random_search = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=param_distributions,
    n_iter=3,          # ULTRA EXPRESS
    scoring="f1",
    cv=cv,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

print("\nEntrenando búsqueda rápida de hiperparámetros...")
random_search.fit(X, y)
print("\n✔ BÚSQUEDA EXPRESS COMPLETADA ✔")

print("\nMejor F1 (CV):", random_search.best_score_)
print("Mejores parámetros encontrados:")
for k, v in random_search.best_params_.items():
    print(f"  {k}: {v}")

# =========================================================
# 6) Evaluación FINAL con el mejor modelo
# =========================================================
best_pipe = random_search.best_estimator_

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

best_pipe.fit(X_train, y_train)
y_pred = best_pipe.predict(X_test)

print("\n📊 CLASSIFICATION REPORT – MEJOR MODELO")
print(classification_report(y_test, y_pred, digits=3))

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=["No Fraude", "Fraude"])
disp.plot(cmap="Blues")
plt.title("Matriz de Confusión – XGBoost + SMOTE (Mejor Modelo)")
plt.show()
